# Model Training

Model training step for the project '**Aftershock pattern prediction based on earthquake rupture data for improved seismic hazard assessment**' (pred_seism_aftXYZ). DeVries18 will refer to the article 'Deep learning of aftershock patterns following large earthquakes' by Phoebe M. R. DeVries, Fernanda Viégas, Martin Wattenberg & Brendan J. Meade, and published in Nature in 2018 (https://www.nature.com/articles/s41586-018-0438-y ).

Inputs from previous steps of the process model:
-  'Features_DeVries18.pkl': Features and target of baseline model (DeVries18), as pickle file;
-  'Features_new.pkl': New features and same target as baseline, as pickle file;
-  'model_baseline_DeVries18_init.h5': Baseline deep neural net of DeVries18, as (untrained) Keras model HDF5 file;
-  'model_baseline_DeVries18_simplified_init.h5': Simplified topology for baseline DNN of DeVries18, as (untrained) Keras model HDF5 file;
-  'model_DNN_init.h5': Deep neural net topology proposed for this project, as (untrained) Keras model HDF5 file;
-  'model_ANN_init.h5': Shallow artificial neural net proposed for this project, as (untrained) Keras model HDF5 file.

For comparison with the DeVries study, we will use the same training and test sets ('Training_FileNames.h5', 'Testing_FileNames.h5'), first imported from the Google Drive https://drive.google.com/drive/folders/1c5Rb_6EsuP2XedDjg37bFDyf8AadtGDa, as given in the README.md of https://github.com/phoebemrdevries/Learning-aftershock-location-patterns.

## Split dataset into training and testing sets

In [ ]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': 'XXXXX',
    'IAM_SERVICE_ID': 'XXXXX',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'predseismaftxyz-donotdelete-pr-dfvzajzxij3spi',
    'FILE': 'Training_FileNames.h5'
}
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_2 = {
    'IBM_API_KEY_ID': 'XXXXX',
    'IAM_SERVICE_ID': 'XXXXX',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'predseismaftxyz-donotdelete-pr-dfvzajzxij3spi',
    'FILE': 'Testing_FileNames.h5'
}

from ibm_botocore.client import Config
import ibm_boto3

#Cloud Object Storage
cos1 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_1['ENDPOINT'])
cos2 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_2['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials_2['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials_2['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_2['ENDPOINT'])

In [1]:
#cos1.download_file(Bucket=credentials_1['BUCKET'], Key='Training_FileNames.h5', Filename='Training_FileNames.h5')
#cos2.download_file(Bucket=credentials_2['BUCKET'], Key='Testing_FileNames.h5', Filename='Testing_FileNames.h5')
!ls

Features_DeVries18.pkl				None0000000.png
Features_new.pkl				pred_seism_aftXYZ
LabelledDataset_DeVries18_balanced.pkl		srcmod2.py
LabelledDataset_DeVries18.pkl			SRCMOD_cleaned.pkl
model_ANN_init.h5				Testing_FileNames.h5
model_baseline_DeVries18_init.h5		TestingSet_X_DeVries18.pkl
model_baseline_DeVries18_simplified_init.h5	TestingSet_X_new.pkl
model_baseline_DeVries18_simplified_trained.h5	TestingSet_y_DeVries18.pkl
model_baseline_DeVries18_trained.h5		TestingSet_y_new.pkl
model_DNN_init.h5				Training_FileNames.h5


In [1]:
import h5py
import numpy as np
import pandas as pd
import sklearn

h5file1 = h5py.File('Training_FileNames.h5', 'r')
training_filenames = np.array(h5file1.get('file_names_training'))
h5file2 = h5py.File('Testing_FileNames.h5', 'r')
testing_filenames = np.array(h5file2.get('file_names_testing'))

training_IDs_temp = map(lambda x: str(x, 'utf-8'), training_filenames)   #from byte to string
training_IDs = list(map(lambda x: 's' + x[0:x.find('_')], training_IDs_temp))  #extract event tag only
testing_IDs_temp = map(lambda x: str(x, 'utf-8'), testing_filenames)   #from byte to string
testing_IDs = list(map(lambda x: 's' + x[0:x.find('_')], testing_IDs_temp))  #extract event tag only

In [2]:
Features_DeVries18 = pd.read_pickle('Features_DeVries18.pkl')
Features_new = pd.read_pickle('Features_new.pkl')

In [3]:
# create training & testing sets
TrainingSet_DeVries18 = Features_DeVries18.loc[Features_DeVries18['ID'].isin(training_IDs)]
TestingSet_DeVries18 = Features_DeVries18.loc[Features_DeVries18['ID'].isin(testing_IDs)]
TrainingSet_new = Features_new.loc[Features_new['ID'].isin(training_IDs)]
TestingSet_new = Features_new.loc[Features_new['ID'].isin(testing_IDs)]

# ratio of training samples
[len(TrainingSet_DeVries18), len(TestingSet_DeVries18), len(TrainingSet_DeVries18)/(len(TrainingSet_DeVries18)+len(TestingSet_DeVries18))]

[164068, 38576, 0.8096366040938789]

In [4]:
# shuffle training sets
TrainingSet_DeVries18 = sklearn.utils.shuffle(TrainingSet_DeVries18)
TrainingSet_new = sklearn.utils.shuffle(TrainingSet_new)

## Train baseline model

In [5]:
TrainingSet_DeVries18.head(10)

,ID,aftershocksyn,posabsxx,posabsxy,posabsyy,posabsxz,posabsyz,posabszz,negabsxx,negabsxy,negabsyy,negabsxz,negabsyz,negabszz
44321,s2011TOHOKU01SATA,0.0,0.261674,0.228577,0.288184,0.011514,0.057211,0.012151,-0.261674,-0.228577,-0.288184,-0.011514,-0.057211,-0.012151
38321,s2011TOHOKU01SATA,1.0,1.117715,0.931841,0.326656,0.851416,0.248543,1.119697,-1.117715,-0.931841,-0.326656,-0.851416,-0.248543,-1.119697
2621,s2006KURILI02SLAD,1.0,0.461496,0.204893,0.304185,0.219381,0.139683,0.062180,-0.461496,-0.204893,-0.304185,-0.219381,-0.139683,-0.062180
9352,s1992LANDER01ZENG,1.0,0.027717,0.136167,0.035541,0.065846,0.055786,0.043923,-0.027717,-0.136167,-0.035541,-0.065846,-0.055786,-0.043923
85015,s2011TOHOKU01SATA,1.0,1.192538,0.346585,0.232770,0.576206,0.159664,0.238143,-1.192538,-0.346585,-0.232770,-0.576206,-0.159664,-0.238143
27406,s2007TOCOPI02BEJA,1.0,0.635618,0.012246,0.021826,0.015297,0.058101,0.402572,-0.635618,-0.012246,-0.021826,-0.015297,-0.058101,-0.402572
200829,s2004SUMATR02RHIE,0.0,1.787140,0.508011,1.373126,0.366103,0.027080,3.409591,-1.787140,-0.508011,-1.373126,-0.366103,-0.027080,-3.409591
7287,s2008WENCHU03FIEL,1.0,3.382321,1.690805,2.540791,2.355867,1.305232,0.008482,-3.382321,-1.690805,-2.540791,-2.355867,-1.305232,-0.008482
26717,s2006KURILI01LAYx,0.0,0.990968,0.047057,0.184090,0.386825,0.042854,0.162403,-0.990968,-0.047057,-0.184090,-0.386825,-0.042854,-0.162403
26442,s1974PERUCE01HART,0.0,0.130845,0.026755,0.023255,0.068965,0.015514,0.025366,-0.130845,-0.026755,-0.023255,-0.068965,-0.015514,-0.025366


In [6]:
features = ['posabsxx', 'posabsxy', 'posabsyy', 'posabsxz', 'posabsyz', 'posabszz',
           'negabsxx', 'negabsxy', 'negabsyy', 'negabsxz', 'negabsyz', 'negabszz']
target = 'aftershocksyn'
x_train = TrainingSet_DeVries18[features]
y_train = TrainingSet_DeVries18[target]
x_test = TestingSet_DeVries18[features]
y_test = TestingSet_DeVries18[target]

# save for next step of the process model
x_test.to_pickle("TestingSet_X_DeVries18.pkl")
y_test.to_pickle("TestingSet_y_DeVries18.pkl")

In [7]:
import keras
from keras.models import load_model

# same as in DeVries18
baselinemodel_DeVries18 = load_model('model_baseline_DeVries18_init.h5')
batch_size = 3500
epochs = 5

history = baselinemodel_DeVries18.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Using TensorFlow backend.


Train on 147661 samples, validate on 16407 samples
Epoch 1/5
147661/147661 [==============================] - 4s 24us/step - loss: 0.6428 - binary_accuracy: 0.6609 - val_loss: 0.6085 - val_binary_accuracy: 0.7196
Epoch 2/5
147661/147661 [==============================] - 4s 25us/step - loss: 0.6073 - binary_accuracy: 0.7034 - val_loss: 0.5974 - val_binary_accuracy: 0.7198
Epoch 3/5
147661/147661 [==============================] - 4s 24us/step - loss: 0.5988 - binary_accuracy: 0.7111 - val_loss: 0.5980 - val_binary_accuracy: 0.7204
Epoch 4/5
147661/147661 [==============================] - 4s 24us/step - loss: 0.5935 - binary_accuracy: 0.7158 - val_loss: 0.5929 - val_binary_accuracy: 0.7203
Epoch 5/5
147661/147661 [==============================] - 4s 26us/step - loss: 0.5907 - binary_accuracy: 0.7182 - val_loss: 0.5922 - val_binary_accuracy: 0.7205


This is the baseline model CV results - we should aim at achieving similar or greater accuracy ($\geq$ 72%) for the new model with new set of features. This will however not assure a same or better generalization than the baseline model (AUC = 85%, see next step of the process model).

In [8]:
baselinemodel_DeVries18.save('model_baseline_DeVries18_trained.h5')

### Side note

Since the Devries18 model uses a relatively small input layer (12 nodes), a simpler DNN topology should do just fine. We show it, only for illustrative purpose. The model proposed in this project will use another set of features.

In [9]:
baselinemodel_DeVries18_simplified = load_model('model_baseline_DeVries18_simplified_init.h5')

history = baselinemodel_DeVries18_simplified.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Train on 147661 samples, validate on 16407 samples
Epoch 1/5
147661/147661 [==============================] - 0s 3us/step - loss: 0.6524 - binary_accuracy: 0.6565 - val_loss: 0.6124 - val_binary_accuracy: 0.7103
Epoch 2/5
147661/147661 [==============================] - 0s 2us/step - loss: 0.6043 - binary_accuracy: 0.7008 - val_loss: 0.5883 - val_binary_accuracy: 0.7117
Epoch 3/5
147661/147661 [==============================] - 0s 2us/step - loss: 0.5927 - binary_accuracy: 0.7079 - val_loss: 0.5836 - val_binary_accuracy: 0.7129
Epoch 4/5
147661/147661 [==============================] - 0s 2us/step - loss: 0.5896 - binary_accuracy: 0.7099 - val_loss: 0.5814 - val_binary_accuracy: 0.7137
Epoch 5/5
147661/147661 [==============================] - 0s 2us/step - loss: 0.5879 - binary_accuracy: 0.7123 - val_loss: 0.5795 - val_binary_accuracy: 0.7140


In [10]:
# note the similar CV accuracy using 12 - 8 - 8 - 1 instead of 12 - 50 - 50 - 50 - 50 - 50 - 50 - 1
baselinemodel_DeVries18_simplified.save('model_baseline_DeVries18_simplified_trained.h5')

## Train new models

Training new models with the proposed set of features based on geometry and kinematics (minimum distance to mainshock rupture and mean slip on rupture).

In [11]:
#features = ['mindist', 'dipMean', 'strikeMean', 'slipMean']   #best 4 - 12 - 12 - 1
features = ['mindist', 'slipMean']   #best 2 - 6 - 6 - 1
target = 'aftershocksyn'
x_train = TrainingSet_new[features]
y_train = TrainingSet_new[target]
x_test = TestingSet_new[features]
y_test = TestingSet_new[target]

# save for next step of the process model
x_test.to_pickle("TestingSet_X_new.pkl")
y_test.to_pickle("TestingSet_y_new.pkl")

### Train new DNN (simplified topology)

In [12]:
model_DNN = load_model('model_DNN_init.h5')

history = model_DNN.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Train on 147661 samples, validate on 16407 samples
Epoch 1/5
147661/147661 [==============================] - 0s 3us/step - loss: 0.6799 - binary_accuracy: 0.5114 - val_loss: 0.6695 - val_binary_accuracy: 0.5169
Epoch 2/5
147661/147661 [==============================] - 0s 1us/step - loss: 0.6638 - binary_accuracy: 0.6333 - val_loss: 0.6536 - val_binary_accuracy: 0.7070
Epoch 3/5
147661/147661 [==============================] - 0s 1us/step - loss: 0.6522 - binary_accuracy: 0.6769 - val_loss: 0.6414 - val_binary_accuracy: 0.7193
Epoch 4/5
147661/147661 [==============================] - 0s 1us/step - loss: 0.6433 - binary_accuracy: 0.6888 - val_loss: 0.6299 - val_binary_accuracy: 0.7331
Epoch 5/5
147661/147661 [==============================] - 0s 1us/step - loss: 0.6318 - binary_accuracy: 0.7032 - val_loss: 0.6160 - val_binary_accuracy: 0.7355


In [ ]:
# Documenting a first DNN model result
# we tried the following topology 4 - 12 - 12 - 1, the 4 features being: ['mindist', 'dipMean', 'strikeMean', 'slipMean']
# we obtained: 
#Epoch 5/5
#147801/147801 [==============================] - 0s 2us/step - loss: 0.6134 - binary_accuracy: 0.6829 - 
#                                                           val_loss: 0.5801 - val_binary_accuracy: 0.7228

# additional tuning involved changes in topology, using relu instead of tanh activation, dropout rate change [NOT SHOWN for clarity].

In [20]:
model_DNN.save('model_DNN_trained.h5')

### Train ANN

Training of an Artificial Neural Network with one hidden layer (no deep learning!).

In [14]:
model_ANN = load_model('model_ANN_init.h5')

history = model_ANN.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Train on 147661 samples, validate on 16407 samples
Epoch 1/5
147661/147661 [==============================] - 0s 2us/step - loss: 0.6613 - binary_accuracy: 0.6469 - val_loss: 0.6447 - val_binary_accuracy: 0.6818
Epoch 2/5
147661/147661 [==============================] - 0s 1us/step - loss: 0.6318 - binary_accuracy: 0.6832 - val_loss: 0.6202 - val_binary_accuracy: 0.6816
Epoch 3/5
147661/147661 [==============================] - 0s 1us/step - loss: 0.6112 - binary_accuracy: 0.6878 - val_loss: 0.6023 - val_binary_accuracy: 0.6924
Epoch 4/5
147661/147661 [==============================] - 0s 1us/step - loss: 0.5946 - binary_accuracy: 0.7048 - val_loss: 0.5874 - val_binary_accuracy: 0.7163
Epoch 5/5
147661/147661 [==============================] - 0s 1us/step - loss: 0.5812 - binary_accuracy: 0.7257 - val_loss: 0.5760 - val_binary_accuracy: 0.7326


In [21]:
model_ANN.save('model_ANN_trained.h5')

### Train XGBoost classifier

To not only test neural networks, we here test another standard machine learning algorithm. XGBoost has become a method of choice in recent Kaggle competitions (althought this does not mean that it will perform better than neural networks in the present case - "No Free Lunch" theorem):

In [16]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

params = {
    'max_depth': [5, 7, 10]
}
gridsearch = GridSearchCV(estimator = xgb.XGBClassifier(
                          objective = "binary:logistic"
                          ),
                        param_grid = params,
                        scoring='accuracy',
                        n_jobs=1,
                        iid=False,
                        cv=5)
gridsearch.fit(x_train, y_train)
gridsearch.grid_scores_, gridsearch.best_params_, gridsearch.best_score_

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


([mean: 0.78089, std: 0.00282, params: {'max_depth': 5},
  mean: 0.78431, std: 0.00303, params: {'max_depth': 7},
  mean: 0.78382, std: 0.00285, params: {'max_depth': 10}],
 {'max_depth': 7},
 0.7843089214705923)

In [17]:
#!pip install joblib

    100% |████████████████████████████████| 286kB 3.5MB/s eta 0:00:01


In [18]:
from joblib import dump

dump(gridsearch, 'model_XGBoost.joblib') 
!ls

Features_DeVries18.pkl				model_XGBoost.joblib
Features_new.pkl				None0000000.png
LabelledDataset_DeVries18_balanced.pkl		pred_seism_aftXYZ
LabelledDataset_DeVries18.pkl			srcmod2.py
model_ANN_init.h5				SRCMOD_cleaned.pkl
model_baseline_DeVries18_init.h5		Testing_FileNames.h5
model_baseline_DeVries18_simplified_init.h5	TestingSet_X_DeVries18.pkl
model_baseline_DeVries18_simplified_trained.h5	TestingSet_X_new.pkl
model_baseline_DeVries18_trained.h5		TestingSet_y_DeVries18.pkl
model_DNN_init.h5				TestingSet_y_new.pkl
model_DNN_trained.h5				Training_FileNames.h5
